In [8]:
#Autor: Andrés
#Tarea: https://github.com/UCM-GIDIA-PD1/c2425-R4/issues/15
#Objetivo: crear particiones de datos para entrener modelos (train, validation, test)
#Descripción y uso: en primer lugar lee el data frame de peleas_ponderadas de la carpeta data se dividen el conjuntos
#de datos en tres subconjuntos y se guardan en parquets. Para usarlo simplemente ejecutar el archivo.
#Estado actual: terminado

In [1]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
ruta = os.path.join(os.getcwd(), "..", "..", "data", "processed", "peleas_ponderadas.parquet")
df = pd.read_parquet(ruta)

In [3]:
df

,DATE,Peleador_A,Peleador_B,WINNER,KD_A,KD_B,SIG_STR_A,SIG_STR_B,TD_PORC_A,TD_PORC_B,...,Derrotas_Sub_A,Derrotas_Sub_B,Derrotas_Decision_A,Derrotas_Decision_B,KD_DIFF,SIG_STR_DIFF,TD_DIFF,SUB_ATT_DIFF,REV_DIFF,CTRL_DIFF
0,2000-06-09,Kevin Randleman,Pedro Rizzo,False,0.0,0.8,0.279,0.315,0.770,0.000,...,0.0,0.0,0.8,0.0,-0.8,-0.036,0.615385,1.0,0.0,526.0
1,2000-09-22,Jeremy Horn,Eugene Jackson,False,0.0,0.5,0.696,0.550,0.084,0.000,...,0.0,0.5,0.0,0.0,-0.5,0.146,0.250000,0.4,0.0,3.3
2,2001-02-23,Tito Ortiz,Evan Tanner,False,0.3,0.0,0.552,0.741,0.874,1.000,...,0.0,0.0,0.0,0.0,0.3,-0.189,0.178571,-0.1,0.0,458.1
3,2001-05-04,Chuck Liddell,Kevin Randleman,False,0.0,0.0,0.508,0.532,0.000,0.758,...,1.0,0.0,0.0,1.0,0.0,-0.024,-0.518519,-1.5,0.2,-455.0
4,2001-06-29,Pat Miletich,Shonie Carter,False,0.0,1.0,0.451,0.521,0.500,0.646,...,0.5,0.0,0.0,0.0,-1.0,-0.070,0.037945,0.8,-1.2,-137.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,2025-02-15,Dricus Du Plessis,Sean Strickland,False,0.2,0.2,0.457,0.455,0.582,0.000,...,0.0,0.0,0.0,3.8,0.0,0.002,0.444444,1.0,0.0,156.0
3472,2025-02-15,Zhang Weili,Tatiana Suarez,False,0.3,0.0,0.654,0.562,0.585,0.659,...,0.0,0.0,1.0,0.0,0.3,0.092,0.211289,0.4,0.2,416.2
3473,2025-02-15,Jake Matthews,Francisco Prado,False,0.0,0.3,0.432,0.452,0.500,0.000,...,3.0,0.0,2.8,1.5,-0.3,-0.020,0.250000,0.0,0.3,78.7
3474,2025-02-15,Jack Jenkins,Gabriel Santos,True,1.0,0.5,0.563,0.606,0.200,0.740,...,0.0,0.0,0.0,1.0,0.5,-0.043,-0.348148,-0.6,0.0,-185.7


In [4]:
df = df[df["DATE"].dt.year >= 2010]

In [5]:
import pandas as pd

# Ordenar por fecha (si no está ordenado ya)
df = df.sort_values(by="DATE")

# Definir los tamaños de cada subconjunto
train_size = 0.8  # 80%
test_size = 0.2    # 20%

# Calcular índice de corte
n = len(df)
train_end = int(n * train_size)

# Dividir el DataFrame
df_train = df.iloc[:train_end]
df_test = df.iloc[train_end:]

# Obtener las fechas de corte
train_end_date = df_train["DATE"].max()
test_end_date = df_test["DATE"].max()

print(f"Última fecha en train: {train_end_date}")
print(f"Última fecha en test: {test_end_date}")

# Mostrar tamaños
print(f"Train: {len(df_train)}, Test: {len(df_test)}")


Última fecha en train: 2022-11-12 00:00:00
Última fecha en test: 2025-02-15 00:00:00
Train: 2554, Test: 639


In [6]:
# Verificar tamaños y balance
def check_distribution(data, name):
    total = len(data)
    class_dist = data["WINNER"].value_counts(normalize=True).round(2)
    print(f"{name}: {total} muestras ({total/len(df):.0%}) | Distribución: {class_dist.to_dict()}")

check_distribution(df_train, "Entrenamiento")
check_distribution(df_test, "Prueba")

Entrenamiento: 2554 muestras (80%) | Distribución: {False: 0.56, True: 0.44}
Prueba: 639 muestras (20%) | Distribución: {False: 0.55, True: 0.45}


In [7]:
base_dir = os.path.join(os.getcwd(), "..", "..", "data")
ruta_partition = os.path.join(base_dir, "P2")

os.makedirs(ruta_partition, exist_ok=True)

df_train.to_parquet(os.path.join(ruta_partition, "train.parquet"), index = False)
df_test.to_parquet(os.path.join(ruta_partition, "test.parquet"), index = False)